In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import os

In [10]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False, # excludes final classfication layer as we will add our own custom layer to classify the dogs
    input_shape=(224, 224, 3) # expect images to be resized to 224 x 224, 3 color channels
)

In [11]:
# the layers of the pre-trained model will not be updated during training
# only train the new layers that are added on top
base_model.trainable = False

In [12]:
# adding custom layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(), # reduces the dimensions of the feature maps
    layers.Dense(128, activation='relu'), # a fully connected layer with 128 units and a ReLU activation function
    layers.Dense(2, activation='softmax')  # 2 classes: Dog 1 and Dog 2
])

In [13]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
# Preparing the Dataset Using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [19]:
train_generator = train_datagen.flow_from_directory(
    os.path.abspath('./data/train/'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 220 images belonging to 2 classes.


In [20]:
model.fit(train_generator, epochs=10)

C:\Users\yenla\AppsByYen\GepperPinger\training\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 825ms/step - accuracy: 0.6249 - loss: 0.6616
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 662ms/step - accuracy: 0.9690 - loss: 0.1339
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 662ms/step - accuracy: 0.9611 - loss: 0.0802
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 610ms/step - accuracy: 0.9845 - loss: 0.0499
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 688ms/step - accuracy: 0.9867 - loss: 0.0361
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9989 - loss: 0.0208   
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 1.0000 - loss: 0.0201   
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 1.0000 - loss: 0.0107   
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.0051
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 0.0121   


In [22]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    os.path.abspath('./data/test/'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 40 images belonging to 2 classes.


In [23]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

C:\Users\yenla\AppsByYen\GepperPinger\training\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 143ms/step - accuracy: 1.0000 - loss: 0.0065
Test Loss: 0.006169213447719812
Test Accuracy: 1.0


In [27]:
model.save(os.path.join(os.path.abspath('./trained_models/'), 'dog_identifier.h5'))